In [1]:
import pickle
import glob
import numpy as np

random_coil_shifts = {'ALA': [52.1, 19.3],
                     'ARG': [55.9, 31.0],
                     'ASN': [53.0, 38.9],
                     'ASP': [53.8, 41.2],
                     'CYS': [58.2, 29.4],
                     'GLN': [55.5, 29.4],
                     'GLU': [56.3, 30.3],
                     'GLY': [45.2, None],
                     'HIS': [55.3, 30.1],
                     'ILE': [60.4,38.7],
                     'LEU': [54.5, 42.5],
                     'LYS': [56.2, 32.8],
                     'MET': [55.4, 33.7],
                     'PHE': [57.2, 40.2],
                     'PRO': [62.6, 31.9],
                     'SER': [58.1, 64.1],
                     'THR': [60.9, 69.7],
                     'TRP': [57.3, 30.4],
                     'TYR': [57.6, 39.4],
                     'VAL': [61.4, 32.8]}


In [2]:
def probability_dist(delta_ca: float, delta_cb: float, amino_acid: str):
    delta_i_ca_rc = random_coil_shifts[amino_acid][0]
    delta_i_cb_rc = random_coil_shifts[amino_acid][1]
    
    
    if amino_acid == 'GLY':
        with open('CaCbfromBMRB/'+amino_acid+'_CA.pkl', 'rb') as f:
            content = pickle.load(f)
        shifts = [[x[7], None] for x in content['data']]
    else:
        with open('CaCbfromBMRB/'+amino_acid+'_CA_CB.pkl', 'rb') as f:
            content = pickle.load(f)
        shifts = [[x[4], x[5]] for x in content]
        
    p_AA = 0
    
    for ca_cb in shifts:
        
        delta_i_ca = ca_cb[0]
        delta_i_cb = ca_cb[1]
        
        alpha = (delta_ca - delta_i_ca_rc) - (delta_i_ca - delta_i_ca_rc)
        
        if delta_cb is None or amino_acid == 'GLY': 
            beta = 0
        else:
            beta = (delta_cb - delta_i_cb_rc) - (delta_i_ca - delta_i_ca_rc)
        
        delta_chem = np.exp(-(alpha**2 + beta**2))

        p_AA += delta_chem
    
    p_AA = p_AA / (np.pi*len(shifts))
    
    return p_AA


def per_AA_probability(delta_ca, delta_cb):
    # Probability of each AA per set of Ca and Cb chemical shifts
    p_all = {}
    normalization_factor = 0

    for AA in random_coil_shifts:
        p_all[AA] = probability_dist(delta_ca, delta_cb, AA)
        normalization_factor += probability_dist(delta_ca, delta_cb, AA)
        
    normalization_factor = 1/normalization_factor
    for AA in p_all:
        p_all[AA] = p_all[AA]*normalization_factor
    
    return p_all

def average_probabilities(exp_dt: dict):
    #Average probabilities of AAs in the set if chemical shifts
    TOTAL = {}
    normalization = 0
    for AA in random_coil_shifts:
        prob = 0
        for pair in exp_dt:
            if 'nan' not in pair:
                print(pair)
                prob += exp_dt[pair][AA]
        TOTAL[AA] = prob
        normalization += prob

    normalization_factor = 1 / normalization
    
    for AA in TOTAL:
        TOTAL[AA] = TOTAL[AA] * normalization_factor
        
    return TOTAL

In [4]:
import pandas as pd
import math
import numpy as np


exp_data = pd.read_csv('test.csv')
#print(exp_data)

exp_dict = {}
for i in range(len(exp_data['Ca'])):
    nme = str(exp_data['index'][i])
    if math.isnan(exp_data['Ca'][i]):
        Ca = None
    else:
        Ca = exp_data['Ca'][i]
    if math.isnan(exp_data['Cb'][i]):
        Cb = None
    else:
        Cb = exp_data['Cb'][i]

    exp_dict[nme] = per_AA_probability(Ca, Cb)

df = pd.DataFrame(exp_dict, index=list(exp_dict['ala'].keys()))
df = df.T

df.to_csv('test_output.csv')

In [5]:
print(df)

               ALA            ARG            ASN            ASP  \
ala   9.989580e-01   3.679403e-16   8.377772e-78   1.045894e-90   
his+  6.666265e-19   3.665813e-01   1.772892e-23   1.955247e-31   
his   4.645015e-24   1.302314e-01   7.488919e-19   8.318719e-26   
asn   8.356842e-73   3.825002e-22   9.324003e-01   2.949853e-02   
asp   6.889397e-90   5.515221e-32   2.265856e-01   5.296710e-01   
leu   6.089547e-90   2.125674e-32   1.794975e-01   5.454428e-01   
met   3.361840e-25   7.452188e-02   2.406282e-18   2.999608e-25   
arg   1.750468e-17   2.844691e-01   1.448777e-27   7.730887e-36   
gln   1.024761e-09   1.544884e-02   8.466012e-41   1.160444e-50   
cys   5.310523e-10   1.676180e-02   1.328949e-40   1.747207e-50   
trp   1.874791e-10   1.350214e-02   4.337924e-40   7.217878e-50   
glu   1.863941e-11   4.163515e-02   8.824004e-38   2.344283e-47   
lys   3.807068e-23   1.451262e-01   2.692445e-20   1.807346e-27   
tyr   2.528661e-37   2.914733e-05   1.221202e-09   5.204213e-1